# 정규화 처리한 데이터 set

In [1]:
import folium
import json
import pandas as pd
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm


import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import fiona

In [2]:
final_data = pd.read_csv("output/그리드 전처리 완료/final_data2.csv")
final_data

,gid,acci_cnt,2030val,과속방지턱,무인카메라,신호등,버스아침승차,버스아침하차,버스저녁승차,버스저녁하차,...,스포츠,음식점,학교,교육,출근시간자전거,퇴근시간자전거,지하철아침승차,지하철아침하차,지하철저녁승차,지하철저녁하차
0,다사542434,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,46,201,0.0,0.0,0.0,0.0
1,다사547430,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,53,137,0.0,0.0,0.0,0.0
2,다사549419,1.0,67.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,2,10,0.0,0.0,0.0,0.0
3,다사551417,1.0,26.0,0.0,0.0,0.0,2486.75,3392.25,1914.833333,3168.333333,...,0.0,1.0,0.0,0.0,3,9,0.0,0.0,0.0,0.0
4,다사564435,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,3.0,0.0,0.0,16,54,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8983,다사559438,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,23,105,0.0,0.0,0.0,0.0
8984,다사618387,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
8985,다사559464,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,135,886,0.0,0.0,0.0,0.0
8986,다사585417,0.0,0.0,0.0,0.0,7.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,1.0,219,303,0.0,0.0,0.0,0.0


In [3]:
final_data = final_data.set_index("gid")
final_data

,acci_cnt,2030val,과속방지턱,무인카메라,신호등,버스아침승차,버스아침하차,버스저녁승차,버스저녁하차,관광여가오락,...,스포츠,음식점,학교,교육,출근시간자전거,퇴근시간자전거,지하철아침승차,지하철아침하차,지하철저녁승차,지하철저녁하차
gid,,,,,,,,,,,,,,,,,,,,,
다사542434,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,46,201,0.0,0.0,0.0,0.0
다사547430,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,53,137,0.0,0.0,0.0,0.0
다사549419,1.0,67.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,2,10,0.0,0.0,0.0,0.0
다사551417,1.0,26.0,0.0,0.0,0.0,2486.75,3392.25,1914.833333,3168.333333,0.0,...,0.0,1.0,0.0,0.0,3,9,0.0,0.0,0.0,0.0
다사564435,1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,3.0,0.0,0.0,16,54,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
다사559438,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,23,105,0.0,0.0,0.0,0.0
다사618387,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
다사559464,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,135,886,0.0,0.0,0.0,0.0


In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_data[:] = scaler.fit_transform(final_data[:])

In [6]:
final_data

,acci_cnt,2030val,과속방지턱,무인카메라,신호등,버스아침승차,버스아침하차,버스저녁승차,버스저녁하차,관광여가오락,...,스포츠,음식점,학교,교육,출근시간자전거,퇴근시간자전거,지하철아침승차,지하철아침하차,지하철저녁승차,지하철저녁하차
gid,,,,,,,,,,,,,,,,,,,,,
다사542434,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.118557,0.089175,0.0,0.0,0.0,0.0
다사547430,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.136598,0.060781,0.0,0.0,0.0,0.0
다사549419,0.5,0.067269,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.005155,0.004437,0.0,0.0,0.0,0.0
다사551417,0.5,0.026104,0.0,0.0,0.000000,0.080009,0.268385,0.186548,0.430229,0.0,...,0.0,0.003984,0.0,0.000000,0.007732,0.003993,0.0,0.0,0.0,0.0
다사564435,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.011952,0.0,0.000000,0.041237,0.023957,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
다사559438,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.059278,0.046584,0.0,0.0,0.0,0.0
다사618387,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
다사559464,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.347938,0.393079,0.0,0.0,0.0,0.0


In [7]:
data = gpd.read_file('data/서울시 격자 인구 데이터/강남서초2030대인구수 - 복사본.geojson')
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [8]:
data.loc[data['2030val'].apply(pd.isna), '2030val'] = 0
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [9]:
# 격자 데이터
data_gid = data[['gid', 'geometry']]

In [10]:
data_gid

,gid,geometry
0,다사579454,"MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,"MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,"MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,"MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,"MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...
9128,다사559438,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [11]:
final_data = pd.merge(final_data, data_gid, on = 'gid', how = 'left')
final_data = gpd.GeoDataFrame(final_data, geometry='geometry')


In [12]:
final_data

,gid,acci_cnt,2030val,과속방지턱,무인카메라,신호등,버스아침승차,버스아침하차,버스저녁승차,버스저녁하차,...,음식점,학교,교육,출근시간자전거,퇴근시간자전거,지하철아침승차,지하철아침하차,지하철저녁승차,지하철저녁하차,geometry
0,다사542434,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.118557,0.089175,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((126.98192 37.48871, 126.98192 ..."
1,다사547430,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.136598,0.060781,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((126.98760 37.48513, 126.98760 ..."
2,다사549419,0.5,0.067269,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.005155,0.004437,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((126.98993 37.47523, 126.98993 ..."
3,다사551417,0.5,0.026104,0.0,0.0,0.000000,0.080009,0.268385,0.186548,0.430229,...,0.003984,0.0,0.000000,0.007732,0.003993,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((126.99221 37.47343, 126.99220 ..."
4,다사564435,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011952,0.0,0.000000,0.041237,0.023957,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.00680 37.48972, 127.00680 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9220,다사559438,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.059278,0.046584,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9221,다사618387,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9222,다사559464,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.347938,0.393079,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9223,다사585417,0.0,0.000000,0.0,0.0,0.233333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.019231,0.564433,0.134428,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [13]:
final_data.to_file('output/그리드 전처리 완료/analysis4_dataset_0524.geojson', driver="GeoJSON",index=False)

## 신호등이 있는 위치만 선정한다

In [1]:
import folium
import json
import pandas as pd
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import statsmodels.api as sm

#import geopandas as gpd
#from shapely.geometry import Polygon, LineString, Point
# import fiona

In [2]:
from matplotlib import font_manager, rc
font_path = "data/THEdog.ttf"   #폰트파일의 위치
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

import matplotlib.pyplot as plt


%matplotlib inline

In [7]:
gid_data = pd.read_csv("output/그리드 전처리 완료/final_data2_정규화.csv")
gid_data

,gid,acci_cnt,2030val,과속방지턱,무인카메라,신호등,버스아침승차,버스아침하차,버스저녁승차,버스저녁하차,...,스포츠,음식점,학교,교육,출근시간자전거,퇴근시간자전거,지하철아침승차,지하철아침하차,지하철저녁승차,지하철저녁하차
0,다사542434,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.118557,0.089175,0.0,0.0,0.0,0.0
1,다사547430,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.136598,0.060781,0.0,0.0,0.0,0.0
2,다사549419,0.5,0.067269,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.005155,0.004437,0.0,0.0,0.0,0.0
3,다사551417,0.5,0.026104,0.0,0.0,0.000000,0.080009,0.268385,0.186548,0.430229,...,0.0,0.003984,0.0,0.000000,0.007732,0.003993,0.0,0.0,0.0,0.0
4,다사564435,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.011952,0.0,0.000000,0.041237,0.023957,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8983,다사559438,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.059278,0.046584,0.0,0.0,0.0,0.0
8984,다사618387,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8985,다사559464,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.347938,0.393079,0.0,0.0,0.0,0.0
8986,다사585417,0.0,0.000000,0.0,0.0,0.233333,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.019231,0.564433,0.134428,0.0,0.0,0.0,0.0


In [8]:
gid_data = gid_data[["gid","acci_cnt","2030val","신호등","버스저녁하차","생활서비스","소매점","음식점","퇴근시간자전거"]]
gid_data

,gid,acci_cnt,2030val,신호등,버스저녁하차,생활서비스,소매점,음식점,퇴근시간자전거
0,다사542434,0.5,0.000000,0.000000,0.000000,0.039216,0.005831,0.000000,0.089175
1,다사547430,0.5,0.000000,0.000000,0.000000,0.058824,0.014577,0.000000,0.060781
2,다사549419,0.5,0.067269,0.000000,0.000000,0.000000,0.002915,0.000000,0.004437
3,다사551417,0.5,0.026104,0.000000,0.430229,0.019608,0.005831,0.003984,0.003993
4,다사564435,0.5,0.000000,0.000000,0.000000,0.039216,0.000000,0.011952,0.023957
...,...,...,...,...,...,...,...,...,...
8983,다사559438,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046584
8984,다사618387,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8985,다사559464,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.393079
8986,다사585417,0.0,0.000000,0.233333,0.000000,0.000000,0.005831,0.000000,0.134428


In [9]:
new_gid_data = gid_data[gid_data["신호등"] > 0.0]
new_gid_data

,gid,acci_cnt,2030val,신호등,버스저녁하차,생활서비스,소매점,음식점,퇴근시간자전거
5,다사566432,0.5,0.009036,0.100000,0.000000,0.000000,0.014577,0.000000,0.017746
8,다사569441,0.5,0.000000,0.233333,0.000000,0.019608,0.002915,0.007968,0.023957
12,다사573440,0.5,0.000000,0.133333,0.000000,0.000000,0.005831,0.019920,0.031500
13,다사574471,0.5,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.312334
15,다사575464,0.5,0.000000,0.533333,0.178802,0.000000,0.000000,0.015936,0.467613
...,...,...,...,...,...,...,...,...,...
8957,다사616373,0.0,0.000000,0.133333,0.001765,0.000000,0.000000,0.000000,0.000000
8974,다사550417,0.0,0.000000,0.033333,0.000000,0.019608,0.000000,0.000000,0.004437
8977,다사619398,0.0,0.000000,0.033333,0.004374,0.000000,0.000000,0.000000,0.000887
8982,다사564421,0.0,0.000000,0.200000,0.056562,0.000000,0.000000,0.000000,0.004437


In [10]:
new_gid_data = new_gid_data.set_index("gid")
new_gid_data

,acci_cnt,2030val,신호등,버스저녁하차,생활서비스,소매점,음식점,퇴근시간자전거
gid,,,,,,,,
다사566432,0.5,0.009036,0.100000,0.000000,0.000000,0.014577,0.000000,0.017746
다사569441,0.5,0.000000,0.233333,0.000000,0.019608,0.002915,0.007968,0.023957
다사573440,0.5,0.000000,0.133333,0.000000,0.000000,0.005831,0.019920,0.031500
다사574471,0.5,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.312334
다사575464,0.5,0.000000,0.533333,0.178802,0.000000,0.000000,0.015936,0.467613
...,...,...,...,...,...,...,...,...
다사616373,0.0,0.000000,0.133333,0.001765,0.000000,0.000000,0.000000,0.000000
다사550417,0.0,0.000000,0.033333,0.000000,0.019608,0.000000,0.000000,0.004437
다사619398,0.0,0.000000,0.033333,0.004374,0.000000,0.000000,0.000000,0.000887


In [11]:
new_gid_data.to_csv("output/그리드 전처리 완료/final4_0524.csv", encoding='utf-8-sig')